In [167]:
import numpy as np
import pandas as pd
from pyinform import *

In [168]:
SectorReturns = pd.read_csv("NormalisedSectorReturns.csv", index_col=0)
print(SectorReturns.shape)
SectorReturns.head()

(5291, 10)


,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
DF = SectorReturns.dropna()
DF.head()

,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-02-25,0.598655,-0.362595,-1.813326,-0.608180,-0.314499,-1.131099,-1.320868,-2.456540,0.725513,-1.709158
1992-02-26,1.482999,-0.536391,-1.203702,0.689234,1.158635,-0.013335,-0.304987,-1.497377,1.245510,0.459472
1992-02-27,1.932779,0.151317,0.188633,-0.021304,0.493030,-0.020952,-0.525962,-1.382264,0.621771,0.212791
1992-02-28,2.264313,0.003429,0.687197,0.274232,0.662564,-0.456263,-0.332417,-1.703039,0.413180,-0.870169
1992-03-02,1.119821,1.108068,1.012923,0.143172,0.636739,-0.686971,-0.686194,-1.379845,0.883358,-1.242292


# Get Quantile-Binning

In [170]:
q = 100
QDF = pd.DataFrame()

for i in DF.columns:
    ts = DF[i]
    QDF[i] = pd.qcut(ts, q, labels=range(q))

def QuantileBin(DF, q):
    out = pd.DataFrame()

    for i in DF.columns:
        x = DF[i]
        out[i] = pd.qcut(x, q, labels = range(q))
    return(out)

QDF = QuantileBin(DF, 100)

# Transfer Entropy

Via package pyinform in https://www.researchgate.net/publication/323135385_Inform_A_toolkit_for_information-theoretic_analysis_of_complex_systems

cite at
https://ieeexplore.ieee.org/document/8285197

Documentation:
https://elife-asu.github.io/PyInform/_modules/pyinform/transferentropy.html

In [171]:
def CausalTE(QDF, k=1):
    m = QDF.shape[1]
    Causality = np.zeros((m,m))

    for i in range(m):
        for j in range(m):
            I, J = QDF.columns[i], QDF.columns[j]
            Cij = transfer_entropy(source=QDF[I],
                            target = QDF[J], k=k)
            Causality[i,j] = Cij
    return(Causality)

In [172]:
Causaliy = CausalTE(QDF)

# Random Transfer Entropy

In [173]:
def Random(DF):

    out = pd.DataFrame()

    for i in DF.columns:
        out[i] = np.random.permutation(DF[i].values)

    out.index = DF.index
    return(out)

In [174]:
Random(QDF)

,Energy,Materials,Financials,Consumer Staples,Consumer Discretionary,Utilities,Telecommunications Services,Information Technology,Industrials,Health Care
Date,,,,,,,,,,
1992-02-25,39,46,13,26,57,23,19,30,35,79
1992-02-26,20,31,9,12,40,80,38,46,64,50
1992-02-27,97,55,5,98,47,14,72,2,27,90
1992-02-28,55,0,93,23,58,27,58,79,0,11
1992-03-02,54,65,86,81,98,8,57,47,51,72
...,...,...,...,...,...,...,...,...,...,...
2012-12-24,99,20,13,8,11,12,39,13,34,33
2012-12-26,0,41,73,98,44,3,94,68,43,20
2012-12-27,29,30,87,37,28,45,57,31,77,94


In [175]:
CausalTE(Random(QuantileBin(DF, 100)))

array([[0.        , 4.78549286, 4.77152713, 4.78135479, 4.78589007,
        4.79745086, 4.79557883, 4.79382375, 4.79615133, 4.80966785],
       [4.80022507, 0.        , 4.7775228 , 4.77938931, 4.79378183,
        4.77755251, 4.80072513, 4.78320003, 4.76889216, 4.77733353],
       [4.79005024, 4.78248487, 0.        , 4.79554032, 4.80334531,
        4.78628599, 4.78560041, 4.80378103, 4.78846363, 4.78030594],
       [4.7861107 , 4.7701938 , 4.7845059 , 0.        , 4.80218421,
        4.78822611, 4.77007262, 4.78383852, 4.77314413, 4.76134759],
       [4.79445978, 4.7891809 , 4.79534389, 4.80209404, 0.        ,
        4.76695252, 4.78759901, 4.79060489, 4.76064641, 4.78944344],
       [4.80979619, 4.78102157, 4.78752575, 4.79815791, 4.77775528,
        0.        , 4.78539377, 4.79013147, 4.78781493, 4.78335972],
       [4.81296339, 4.80650065, 4.78836583, 4.78348206, 4.79875625,
        4.78770023, 0.        , 4.80439157, 4.77152161, 4.80047282],
       [4.80084584, 4.78017466, 4.7934512

In [176]:
def CompareRandom(DF, n_P, q):
    RTE = CausalTE(Random(QuantileBin(DF, q)))

    for i in range(1,n_P):

        RTE +=CausalTE(Random(QuantileBin(DF, q)))

    return(RTE/float(n_P))

In [177]:
np.random.seed(12345)


In [178]:
EnsembleRTE = CompareRandom(DF, 100, 100)

In [179]:
RelevantTE = Causality - RTE

In [180]:
RelevantTE.round(4)

array([[ 0.    , -0.5122, -0.4346, -0.322 , -0.4628, -0.4137, -0.3706,
        -0.4463, -0.4661, -0.4017],
       [-0.4702,  0.    , -0.524 , -0.4041, -0.6209, -0.4507, -0.4169,
        -0.5277, -0.6986, -0.4447],
       [-0.431 , -0.5612,  0.    , -0.4562, -0.6477, -0.4618, -0.4424,
        -0.4921, -0.6605, -0.5128],
       [-0.3848, -0.5061, -0.5229,  0.    , -0.5662, -0.462 , -0.4327,
        -0.4505, -0.5573, -0.5492],
       [-0.4427, -0.637 , -0.6228, -0.4826,  0.    , -0.4499, -0.4244,
        -0.5983, -0.764 , -0.5202],
       [-0.4221, -0.4971, -0.4706, -0.4059, -0.4775,  0.    , -0.4698,
        -0.4356, -0.4817, -0.4356],
       [-0.3865, -0.4757, -0.4661, -0.3897, -0.4669, -0.4804,  0.    ,
        -0.4492, -0.4723, -0.4263],
       [-0.4424, -0.5691, -0.4915, -0.388 , -0.615 , -0.4214, -0.4231,
         0.    , -0.6256, -0.4629],
       [-0.4519, -0.7165, -0.6481, -0.4827, -0.7668, -0.4563, -0.4331,
        -0.6107,  0.    , -0.5223],
       [-0.4188, -0.501 , -0.5367, -0